# Example workflow

In [1]:
import rectanglepy as rectangle
from anndata import AnnData

## Load tutorial data

In [2]:
sc_counts, annotations, bulks  = rectangle.load_tutorial_data()

In [3]:
sc_counts.iloc[:, :10].head()

,MIR1302-2HG,AL627309.1,AL627309.4,AC114498.1,AL669831.5,FAM41C,AL645608.5,NOC2L,PERM1,ISG15
E2L4_GATGCTACAGGCACAA,0,0,0,0,0,0,0,0,0,7
L5_AACAACCAGGAACTAT,0,0,0,0,0,0,0,0,0,1
L5_TCCTTCTGTACTCCGG,0,0,0,0,0,0,0,0,0,0
L2_GCCCGAACACGTATAC,0,0,0,0,0,0,0,0,0,2
E2L2_ATGCATGTCACACCCT,0,0,0,0,0,1,0,0,0,1


In [4]:
annotations.head()

E2L4_GATGCTACAGGCACAA    Monocytes
L5_AACAACCAGGAACTAT      Monocytes
L5_TCCTTCTGTACTCCGG      Monocytes
L2_GCCCGAACACGTATAC      Monocytes
E2L2_ATGCATGTCACACCCT    Monocytes
Name: 0, dtype: object

In [5]:
bulks.iloc[:, :10].head()

,UBE2Q2P2,SSX9,CXorf67,EFCAB8,SPATA31B1P,SDR16C6P,GTPBP6,EFCAB12,A1BG,A1CF
pbmc_1,0.000000,0.0,0.118865,0.00000,0.0,0.0,57.280197,0.252188,10.863071,0.0
pbmc_10,0.081115,0.0,0.086782,0.00000,0.0,0.0,72.275326,0.095742,14.785649,0.0
pbmc_12,0.000000,0.0,0.188464,0.03157,0.0,0.0,50.997427,0.100608,7.492537,0.0


### Convert to AnnData object

In [7]:
sc_adata = AnnData(sc_counts, obs=annotations.to_frame(name="cell_type"))

# Single step Rectangle workflow

In [ ]:
estimations, signature_result = rectangle.rectangle(sc_adata, bulks)

In [9]:
estimations

,B cells,ILC,Monocytes,NK cells,Plasma cells,Platelet,T cells CD4 conv,T cells CD8,Tregs,mDC,pDC,Unknown
pbmc_1,0.075919,0.001291,0.279064,0.046658,0.010834,0.008950,0.139600,0.268706,0.069930,0.017415,0.001971,0.079661
pbmc_10,0.093986,0.003410,0.316445,0.033528,0.002279,0.036153,0.243435,0.146063,0.011599,0.032518,0.005273,0.075309
pbmc_12,0.060533,0.017572,0.251589,0.114043,0.002038,0.027041,0.119488,0.325029,0.024586,0.027099,0.004422,0.026560


signature_result is a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object. Results of estimations vary slightly from machine to machine (due to difference in theunderlying linear algebra packages)

## Even sampling and consensus

We recommend even subsampling of the sc data.
You can use the `sample_size` parameter to sample a fixed number of cells per cell type,  and run the consensus multiple times to get a more robust result.

In [ ]:
estimations, signature_result = rectangle.rectangle(sc_adata, bulks, consensus_runs=5, sample_size=1500)

# 2-step Rectangle workflow

Rectangle can also be run in two steps, first creating the signature and then deconvoluting the bulk data.

## Create Signature result

In [ ]:
signature_result = rectangle.pp.build_rectangle_signatures(sc_adata, bulks=bulks)

This creates a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object.

## Deconvolute bulk data

In [ ]:
estimations = rectangle.tl.deconvolution(signature_result, bulks)

This returns a pandas DataFrame with the estimated cell type proportions.